# setup

In [1]:
include("DelayedMatrices.jl")
include("TurtleSearch.jl")
using .DelayedMatrices, .TurtleSearch

In [2]:
include("eom.jl")
using .GoldbergerWiseEoM

In [3]:
using Plots, LaTeXStrings

In [4]:
function paramsearch(;l2=nothing, g2=nothing, FP = 1., φP = 0)
    if g2 != nothing && l2 == nothing
        function paramsearch_l2_m2(l2, m2)
            params = (m2, l2, g2)
            return errBCwithφ(FP, params, φP = φP )
        end
        return paramsearch_l2_m2
    elseif g2 == nothing && l2 != nothing
        function paramserch_g2_m2(g2, m2)
            params = (m2, l2, g2)
            return errBCwithφ(FP, params, φP = φP)
        end
        return paramserch_g2_m2
    end
end

paramsearch (generic function with 1 method)

# $m^2 -\gamma^2$ plot 
- mass in M_IR units
- gamma in 4k+2u units
- x axis: mass
- y axis: gamma
- set up varphi value at the Plank brane(to be zero)
`paramsearch_g2_m2_tmp = paramsearch(l2=l2)`

In [28]:
# 定义计算 g2s0 和 ms0 的函数
function calculate_g2s0_ms0(l2, yₘ, u, k)
    g2s0 = exp10.(range(log10(4k+2u +1e-3), 3, 100))
    ms0 = sqrt.(4l2*(2k+u)*u^2/(3k)*(1-exp(2k*yₘ))/(1-exp((4k+2u)*yₘ)) * (1 .- (4k+2u) ./g2s0))
    return g2s0, ms0
end

# 定义绘制图像的函数
function plot_figure(g2s0, ms0, M_IR, g0, modename, linestyle=:solid)
    plot!(g2s0./g0, ms0./M_IR, alpha=0.8, c=:black, label=modename, line=linestyle, lw=1, xaxis=:log10, yaxis=:log10)
end

# 定义主函数
function plot_main()
    plot()
    let l2s = [1e-3], xrange = (0,3), yrange = (-9.7,-3), logscaled = true
        g0 = let yₘ = π, u = 1e-1, k=37u, g0=4k+2u
            g2s0, ms0 = calculate_g2s0_ms0(l2s[1], yₘ, u, k)
            plot_figure(g2s0, ms0, M_IR, g0, L"m({\mathcal{O}(l^2, \gamma^2)})", :dash)
            4k+2u
        end
        for il2 in eachindex(l2s)
            l2 = l2s[il2]
            paramsearch_g2_m2_tmp = paramsearch(l2=l2)
            m = DelayedMatrix(zeros(50,50), paramsearch_g2_m2_tmp,xrange, yrange, logscaled = logscaled)
            poses = findfirstseed!(m, CartesianIndex(1, 1), CartesianIndex(1, 2))
            @assert poses != nothing "no seed found"
            modenames = ("radion mode", nothing, "KK modes", nothing)
            for i=1:2
                t = Turtles(max(poses...))
                inds = bisearch(t, m)
                xs = [m.xaxis(inds[1])]
                ys = [m.yaxis(inds[2])]
                println((xs[1], ys[1]))
                while nothing != next!(t, m)
                    inds = bisearch(t, m)
                    push!(xs,m.xaxis(inds[1]))
                    push!(ys,m.yaxis(inds[2]))
                end
                plot_figure(xs, ys, M_IR, g0, modenames[i])
                poses = findfirstseed!(m, map(x->x+CartesianIndex(0,1), poses)...)
            end
        end
    end
    f = plot!(ylabel=L"m/M_{IR}", xlabel=L"\gamma^2/(4k+2u)",  dpi=300)
    savefig(f, "figs/paramsearch_gm_m0123_IR_reversedXY.png")
end


plot_main (generic function with 1 method)

In [8]:
# 定义计算 g2s0 和 ms0 的函数
function calculate_g2s0_ms0(l2, yₘ, u, k)
    g2s0 = exp10.(range(log10(4k+2u +1e-3), 1.5, 100))
    ms0 = sqrt.(4l2*(2k+u)*u^2/(3k)*(1-exp(2k*yₘ))/(1-exp((4k+2u)*yₘ)) * (1 .- (4k+2u) ./g2s0))
    return g2s0, ms0
end

# 定义绘制图像的函数
function plot_figure(g2s0, ms0, M_IR, g0, modename, linestyle=:solid)
    plot!(g2s0./g0, ms0./M_IR, alpha=0.8, c=:black, label=modename, line=linestyle, lw=1, xaxis=:log10, yaxis=:log10)
end

# 定义主函数
function plot_main()
    plot()
    let l2s = [1e-3], xrange = (0.8,1.5), yrange = (-9.7,-3), logscaled = true, φP = 1e-1
        g0 = let yₘ = π, u = 1e-1, k=37u, g0=4k+2u
            g2s0, ms0 = calculate_g2s0_ms0(l2s[1], yₘ, u, k)
            plot_figure(g2s0, ms0, M_IR, g0, L"m({\mathcal{O}(l^2, \gamma^2)})", :dash)
            4k+2u
        end
        for il2 in eachindex(l2s)
            l2 = l2s[il2]
            paramsearch_g2_m2_tmp = paramsearch(l2=l2, φP=φP)
            m = DelayedMatrix(zeros(30,30), paramsearch_g2_m2_tmp,xrange, yrange, logscaled = logscaled)
            poses = findfirstseed!(m, CartesianIndex(1, 1), CartesianIndex(1, 2))
            @assert poses != nothing "no seed found"
            modenames = ("radion mode", nothing, "KK modes", nothing)
            for i=1:1
                t = Turtles(max(poses...))
                inds = bisearch(t, m)
                xs = [m.xaxis(inds[1])]
                ys = [m.yaxis(inds[2])]
                println((xs[1], ys[1]))
                while nothing != next!(t, m)
                    inds = bisearch(t, m)
                    push!(xs,m.xaxis(inds[1]))
                    push!(ys,m.yaxis(inds[2]))
                end
                plot_figure(xs, ys, M_IR, g0, modenames[i])
                poses = findfirstseed!(m, map(x->x+CartesianIndex(0,1), poses)...)
            end
        end
    end
    f = plot!(ylabel=L"m/M_{IR}", xlabel=L"\gamma^2/(4k+2u)",  dpi=300)
    savefig(f, "figs/paramsearch_gm_m0123_IR_reversedXY.png")
end


plot_main (generic function with 1 method)

In [9]:
plot_main()

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 (6.309573444801933, 1.907607207870774e-8)
0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

...

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

 0.0 

"/Users/outk/Documents/Codes/Julia/22-7/spectraShooter/figs/paramsearch_gm_m0123_IR_reversedXY.png"